In [87]:
import numpy as np
import pyphi
from pprint import pprint 
from collections import Counter

In [3]:
tpm = np.array([
     [0, 0, 0],
     [0, 0, 1],
     [1, 0, 1],
     [1, 0, 0],
     [1, 1, 0],
     [1, 1, 1],
     [1, 1, 1],
     [1, 1, 0]
 ])

cm = np.array([
     [0, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 1],
     [1, 0, 0, 0, 0, 0],
 ])

labels = ('A', 'B', 'C')
network = pyphi.Network(tpm, cm=cm, node_labels=labels)
state = (1, 0, 0)
node_indices = (0, 1, 2)
subsystem = pyphi.Subsystem(network, state, node_indices)

ValueError: Connectivity matrix must be NxN, where N is the number of nodes in the network.

## Boolean perceptron

In [275]:
class Graph():
    def __init__(self, Adjacency_Matrix = None, label = None, Adjacency_List = None):
        ''' Initializes a Graph Structure (Vertex, Edges) given a Connectivity Matrix or Adjacency List.
        We assume all the edge weights are positive. There are two type of degrees : Outbound and Inbound.
        Sets the vertex label if provided either as an ordered list or dictionary (preferred)'''
        
        self.nodes = {}
        self.edges = {}
        self.no_nodes = 0
        self.no_edges =0
        self.node_dictionary = {'state': 0, 'label': None, 'outbound_degree': 0, 'inbound_degree':0 }
        self.removed_node = []
        
        if(Adjacency_Matrix is not None):
            self.no_nodes = Adjacency_Matrix.shape[0]
            self.no_edges = np.count_nonzero(Adjacency_Matrix)
            self.nodes = {key: self.node_dictionary.copy() for key in range(self.no_nodes)}
        
            for i, row in enumerate(Adjacency_Matrix):
                nonzero_indices = np.nonzero(row)
                self.edges[i] = dict(zip(nonzero_indices[0], row[nonzero_indices[0]]))
                self.nodes[i]['outbound_degree'] = len(nonzero_indices[0])
                self.nodes[i]['inbound_degree'] = len(np.nonzero(Adjacency_Matrix[:,i])[0])


        elif(Adjacency_List is not None):
            # Assuming Adjacency matrix is a dictionary and key are vertices starting from 0 and elements are (node, weight)
            self.no_nodes = len(Adjacency_List)
            self.nodes = {key: self.node_dictionary.copy() for key in range(self.no_nodes)}
            self.edges = Adjacency_List
            inbound_degrees = []
            
            for i, v in enumerate(Adjacency_List.values()):
                self.no_edges += len(v)
                self.nodes[i]['outbound_degree'] = len(v)
                inbound_degrees.extend(list(v.keys()))
                
            inbound_degrees = Counter(inbound_degrees)
            for key in inbound_degrees.keys():
                self.nodes[key]['inbound_degree'] = inbound_degrees[key]
            
            
        # If the label list is given and all the elements are unique 
        if(label is not None and len(set(label)) == len(self.nodes)):
            for i in range(self.no_nodes):
                self.nodes[i]['label'] = label[i]
            
            
    def print_graph(self):
        print("Vertices: \n", self.nodes.keys(), "\n\n")
        print("Edegs:")
        pprint(self.edges)
        
            
    def add_node(self, label = None):
        ''' Creates a new vertex and returns its index'''
        self.no_nodes += 1
        
        if(len(self.removed_node)!=0):
            index = self.removed_node.pop(0)
            self.nodes[index] = self.node_dictionary
            self.nodes[index]['label'] = label
            self.edges[index] = {}
            return index
            
        else:
            self.nodes[self.no_nodes-1] = self.node_dictionary
            self.nodes[self.no_nodes-1]['label'] = label
            self.edges[self.no_nodes-1] = {}
            return self.no_nodes
        
        
    def add_edge(self, vertex_1, vertex_2, weight):
        if(vertex_1 not in self.nodes or vertex_2 not in self.nodes):
            print("One of the Vertices is out of bound")
            return 0
        
        elif(vertex_2 in self.edges[vertex_1]):
            print("Edge is already present")
            return 0
        
        self.edges[vertex_1][vertex_2] = weight
        self.nodes[vertex_1]['outbound_degree'] += 1
        self.nodes[vertex_2]['inbound_degree'] += 1
        self.no_edges += 1
        return 1
    
    '''
    def add_nodes_from(self, list_vertex):
        i = 0
        length_list = len(list_vertex)
        
        for node in self.removed_node:
            self.nodes[node] = self.node_dictionary
    '''
            
        
    def add_edges_from(self, list_edges):
        for i,j, weight in list_edges:
            if((i in self.nodes and j in self.nodes) and (j not in self.edges[i] and weight!=0)):
                self.edges[i][j] = weight
                self.nodes[i]['outbound_degree'] += 1
                self.nodes[j]['inbound_degree'] += 1
                self.no_edges += 1
    
    
    def remove_node(self, vertex):
        if(vertex not in self.nodes):
            print("Vertex not present")
            return 0
        
        for i in self.nodes: # deleting inbound_edges
            if(vertex in self.edges[i].keys()): 
                del self.edges[i][vertex]
        
        del self.edges[vertex] # deleting outbound_edges
        self.no_edges -= (self.nodes[vertex]['inbound_degree'] + self.nodes[vertex]['outbound_degree'])
        self.no_nodes -= 1
        self.removed_node.append(vertex)
        del self.nodes[vertex]
        return 1
    
    def remove_edge(self, vertex1, vertex2):
        if(vertex1 not in self.nodes or vertex2 not in self.nodes):
            print("One of the Vertices is out of bound")
            return 0
        
        elif(vertex2 not in self.edges[vertex1]):
            print("Edge not present")
            return 0
        
        del self.edges[vertex1][vertex2]
        self.nodes[vertex1]['outbound_degree'] -= 1
        self.nodes[vertex2]['inbound_degree'] -= 1
        self.no_edges -= 1
        return 1
    
        
    def node_neighbours(self, vertex):
        return self.edges[vertex].keys()
    
    def change_label(self, vertex, label):
        if(vertex in self.nodes):
            self.nodes[vertex]['label'] = label
            return 1
        
        else:
            print("Vertex out of bound")
            return 0

In [244]:
nonzero_indices = np.nonzero(cm[0])
dict(zip(nonzero_indices[0], cm[0][nonzero_indices[0]]))

{}

In [245]:
cm = np.array([
     [0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 1],
     [1, 0, 0, 0, 0, 0],
 ])

In [281]:
G = Graph(Adjacency_Matrix=cm, label=['A','B','C','D','E', 'F'])

In [282]:
G.add_node()

7

In [283]:
G.add_edges_from([[6,1,3],[1,0,4]])

In [285]:
G.remove_edge(6,1)

1

In [286]:
G.nodes

{0: {'state': 0, 'label': 'A', 'outbound_degree': 0, 'inbound_degree': 2},
 1: {'state': 0, 'label': 'B', 'outbound_degree': 2, 'inbound_degree': 0},
 2: {'state': 0, 'label': 'C', 'outbound_degree': 1, 'inbound_degree': 0},
 3: {'state': 0, 'label': 'D', 'outbound_degree': 1, 'inbound_degree': 0},
 4: {'state': 0, 'label': 'E', 'outbound_degree': 1, 'inbound_degree': 3},
 5: {'state': 0, 'label': 'F', 'outbound_degree': 1, 'inbound_degree': 1},
 6: {'state': 0, 'label': None, 'outbound_degree': 0, 'inbound_degree': 0}}

In [251]:
G.no_nodes

6

In [287]:
pprint(G.edges)

{0: {}, 1: {0: 4, 4: 1}, 2: {4: 1}, 3: {4: 1}, 4: {5: 1}, 5: {0: 1}, 6: {}}


In [63]:
cl = {0: {3: 1, 4: 1}, 1: {4: 1}, 2: {4: 1}, 3: {4: 1}, 4: {5: 1}, 5: {0: 1}}

In [94]:
G_ = Graph(Adjacency_List=cl)

In [95]:
G_.print_graph()

Vertices: 
 dict_keys([0, 1, 2, 3, 4, 5]) 


Edegs:
{0: {3: 1, 4: 1}, 1: {4: 1}, 2: {4: 1}, 3: {4: 1}, 4: {5: 1}, 5: {0: 1}}


In [96]:
G_.no_nodes

6

In [97]:
G_.nodes

{0: {'state': 0, 'label': None, 'outbound_degree': 2, 'inbound_degree': 1},
 1: {'state': 0, 'label': None, 'outbound_degree': 1, 'inbound_degree': 0},
 2: {'state': 0, 'label': None, 'outbound_degree': 1, 'inbound_degree': 0},
 3: {'state': 0, 'label': None, 'outbound_degree': 1, 'inbound_degree': 1},
 4: {'state': 0, 'label': None, 'outbound_degree': 1, 'inbound_degree': 4},
 5: {'state': 0, 'label': None, 'outbound_degree': 1, 'inbound_degree': 1}}

In [91]:
a = list(G_.edges[0].keys())

In [92]:
b = {3, 5}
Counter(a.union(b))

Counter({3: 1, 4: 1, 5: 1})

In [107]:
1 in [1,2,5]

True